## How to fine-tune chat models

미세 조정은 프롬프트에 맞출 수 있는 것보다 훨씬 더 많은 예제를 학습하여 모델을 개선하고, 다양한 작업에서 더 나은 결과를 얻을 수 있도록 합니다. 이 노트북은 새로운 GPT-4o 미니 미세 조정에 대한 단계별 가이드를 제공합니다. 다양한 레시피와 각각에 대한 추출된 일반 재료 목록을 제공하는 RecipeNLG 데이터 세트를 사용하여 엔터티 추출을 수행합니다 . 이는 명명된 엔터티 인식(NER) 작업에 공통적인 데이터 세트입니다.

참고: GPT-4o mini 미세 조정은 Tier 4 및 5 사용 계층 의 개발자에게 제공됩니다 . 미세 조정 대시보드를 방문하여 "생성"을 클릭하고 기본 모델 드롭다운에서 "gpt-4o-mini-2024-07-18"을 선택하여 GPT-4o mini 미세 조정을 시작할 수 있습니다.

다음 단계를 살펴보겠습니다.

- 설정: 데이터 세트를 로드하고 미세 조정할 하나의 도메인으로 필터링합니다.
- 데이터 준비: 훈련 및 검증 사례를 만들고 이를 Files엔드포인트에 업로드하여 미세 조정을 위한 데이터를 준비합니다.
- 미세 조정: 미세 조정된 모델을 만듭니다.
- 추론: 미세 조정된 모델을 사용하여 새로운 입력에 대한 추론을 수행합니다.
이 과정을 마치면 미세 조정된 gpt-4o-mini-2024-07-18모델을 훈련, 평가하고 배포할 수 있게 됩니다.

미세 조정에 대한 자세한 내용은 설명서 가이드 나 API 참조를 참조하세요 .

In [ ]:
!pip install openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 313.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


chat completion API 기반 prompt를 이용한 모델 생성

In [ ]:
# zero-shot
from openai import OpenAI

client = OpenAI(api_key='')

response = client.chat.completions.create(
    model ='gpt-4o-mini-2024-07-18',
    messages =[
    {"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."},
    {"role": "user", "content": 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
    ]
    )


print(response.choices[0].message.content)

- Brown sugar
- Evaporated milk
- Vanilla
- Nuts
- Butter or margarine
- Shredded rice biscuits


In [ ]:
# few-shot
from openai import OpenAI

client = OpenAI(api_key='')

response = client.chat.completions.create(
    model ='gpt-4o-mini-2024-07-18',
    messages =[
    {"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."},
    {"role": "user", "content": 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
    {'role':'assistant','content':'["butter","vanilla","milk","brown sugar"]'},
    {"role": "user", "content": 'Title: Classic Chocolate Chip Cookies\n\nIngredients: ["1 c. unsalted butter", "3/4 c. granulated sugar", "3/4 c. packed brown sugar", "1 tsp. vanilla extract", "2 large eggs", "2 1/4 c. all-purpose flour", "1/2 tsp. baking soda", "1/2 tsp. salt", "2 c. chocolate chips"]\n\nGeneric ingredients: '}
    ]
    )


print(response.choices[0].message.content)

["butter","sugar","vanilla","eggs","flour","baking soda","salt","chocolate chips"]


In [ ]:
import os
import json
import openai
from pprint import pprint

파인 튜닝(fine-tuning)은 특정 도메인에 집중할 때 가장 효과적입니다. 모델이 학습할 수 있도록 데이터셋이 충분히 집중되어 있어야 하지만, 새로운 예시를 놓치지 않도록 어느 정도 일반성도 유지하는 것이 중요합니다. 이를 염두에 두고, 우리는 RecipesNLG 데이터셋에서 www.cookbooks.com의 문서만 포함하는 하위 집합을 추출했습니다.

In [ ]:
import pandas as pd
recipe_df = pd.read_csv('/content/drive/MyDrive/kdt_240424/m9_LLM/data/cookbook_recipes_nlg_10k.csv')

recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## Data preparation

우리는 데이터를 준비하는 것부터 시작할 것입니다. ChatCompletion 형식으로 파인 튜닝할 때, 각 학습 예시는 메시지들의 단순한 목록으로 구성됩니다. 예를 들어, 하나의 항목은 다음과 같이 생겼을 수 있습니다:

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```


훈련 과정에서 이 대화는 나뉘게 되며, 마지막 항목은 모델이 생성할 답변(완성)이 되고, 나머지 메시지들은 프롬프트 역할을 합니다. 따라서 학습 예제를 만들 때 이를 고려해야 합니다. 모델이 여러 차례 주고받는 대화에서 작동할 경우, 대화가 확장되더라도 성능이 떨어지지 않도록 대표적인 예시들을 제공해야 합니다.


In [ ]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def create_user_message(row):
    return f"""TItle:{row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients:"""

def prepare_example_conversation(row):
    messages = []
    messages.append({'role':'system','content':system_message})

    user_message = create_user_message(row)
    messages.append({'role':'user','content':user_message})

    messages.append({'role':'assistant','content':row['NER']})

    return {'messages':messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'TItle:No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients:',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


이제 훈련 데이터로 사용할 데이터 세트의 하위집합에 대해 이 작업을 수행해보겠습니다. 30~50개의 잘 정리된 예제로 시작할 수도 있습니다. 훈련 세트의 크기를 늘리면 성능이 선형적으로 계속 확장되는 것을 볼 수 있지만 작업에 걸리는 시간도 길어집니다

In [ ]:


training_df = recipe_df.loc[:30]

training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'TItle:No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients:'}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'TItle:Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients:'}, {'role': 'assistant', 'content': '["beef", 

훈련 데이터 외에 선택적으로 검증 데이터를 제공할 수도 있습니다. 검증 데이터는 모델이 훈련 세트에 과도하게 적합하지 않은지 확인하는 데 사용됩니다.

In [ ]:
validation_df = recipe_df.loc[20:30]

validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

그럼 다음 각 줄이 하나의 훈련 대화 사례가 되도록 jsonl 파일로 데이터를 저장해야합니다


- JSONL은 파일의 각 줄에 하나의 JSON 객체를 저장하는 형식입니다. 대규모 데이터를 처리할 때 유용하며, 한 줄씩 데이터를 읽거나 쓸 수 있기 때문에 효율적입니다.

In [ ]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename,'w') as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + '\n' # json.dumps(ddict)는 딕셔너리 ddict를 JSON 형식의 문자열로 변환
            out.write(jout)


In [ ]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)


validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [ ]:
!ls

drive  sample_data  tmp_recipe_finetune_training.jsonl	tmp_recipe_finetune_validation.jsonl


In [ ]:
!head -n 5 tmp_recipe_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "TItle:No-Bake Nut Cookies\n\nIngredients: [\"1 c. firmly packed brown sugar\", \"1/2 c. evaporated milk\", \"1/2 tsp. vanilla\", \"1/2 c. broken nuts (pecans)\", \"2 Tbsp. butter or margarine\", \"3 1/2 c. bite size shredded rice biscuits\"]\n\nGeneric ingredients:"}, {"role": "assistant", "content": "[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]"}]}
{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "TItle:Jewell Ball'S Chicken\n\nIngredients: [\"1 small jar chipped beef, cut up\", \"4 boned chicken breasts\", \"1 can cream of mushroom soup\", \"1 carton sour cream\"]\n\nGeneric ingredients:"}, {"role": "as

### Fine-tuning

In [ ]:
from openai import OpenAI
MODEL = 'gpt-4o-mini-2024-07-18'

client = OpenAI(api_key = '')

train = client.files.create(
    file=open('tmp_recipe_finetune_training.jsonl','rb'),
    purpose = 'fine-tune'
)

valid = client.files.create(
    file=open('tmp_recipe_finetune_validation.jsonl','rb'),
    purpose='fine-tune'
)

In [ ]:
train

FileObject(id='file-AMsEf1tPciVbzC75wwQWADD4', bytes=18627, created_at=1724293370, filename='tmp_recipe_finetune_training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
valid

FileObject(id='file-nMpIrIWONfWt3nUa0q4jt9wS', bytes=6935, created_at=1724293370, filename='tmp_recipe_finetune_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
    training_file=train.id,
    validation_file = valid.id,
    model=MODEL,
    hyperparameters={
        'n_epochs':3
    }
)

FineTuningJob(id='ftjob-uzhiyf7eRhSfw9Otc9opRz4o', created_at=1724303996, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-WuW8VroLyUVT2O2Rty6QxV5g', result_files=[], seed=922157045, status='validating_files', trained_tokens=None, training_file='file-PAjniVmsa3ouZz576nBQlfIG', validation_file='file-s5XYbEij1TYcNMTsqrH14dF3', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
response = client.fine_tuning.jobs.retrieve('ftjob-uzhiyf7eRhSfw9Otc9opRz4o')

status = response.status

if status =='succeeded':
    print('작업이 완료되었습니다')

else:
    print(f"작업 상태: {status}")

total_tokens = response.trained_tokens
print(f'사용된 총 토큰 수: {total_tokens}')

작업이 완료되었습니다
사용된 총 토큰 수: 12792


In [ ]:
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-uzhiyf7eRhSfw9Otc9opRz4o', created_at=1724303996, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::9yudTkS6', finished_at=1724304413, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-WuW8VroLyUVT2O2Rty6QxV5g', result_files=['file-9psE7sjEuSlCuNGca4JyhjO3'], seed=922157045, status='succeeded', trained_tokens=12792, training_file='file-PAjniVmsa3ouZz576nBQlfIG', validation_file='file-s5XYbEij1TYcNMTsqrH14dF3', estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

### Full Validation Loss의 목적:
- 과적합 방지: 훈련 데이터에 대한 손실(training loss)은 낮을 수 있지만, 검증 데이터에 대한 성능이 떨어지면 모델이 과적합되고 있는 신호입니다. Full validation loss는 이런 과적합 여부를 감지하는 데 유용합니다.
- 모델 선택: 파인튜닝 중 다양한 하이퍼파라미터 조합을 실험할 때, Full validation loss가 가장 낮은 지점에서 최적의 모델을 선택하는 기준이 될 수 있습니다.

Full validation loss는 파인튜닝 중 모델이 검증 데이터셋에 대해 얼마나 잘 예측하는지를 평가하는 중요한 지표로, 모델의 성능과 일반화 능력을 판단하는 데 도움을 줍니다.

### Inference

마지막 단계는 추론을 위해 미세 조정된 모델을 사용하는 것입니다. FineTuning 클래식과 유사하게 모델 매개변수를 채우는 새로운 미세 조정된 모델 이름으로 `ChatCompletions`을 호출하기만 하면 됩니다

In [ ]:
test_df = recipe_df.loc[30:40]
test_row = test_df.iloc[0]

test_messages = []
test_messages.append({'role':'system','content':system_message})

user_message = create_user_message(test_row)
test_messages.append({'role':'system','content':create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'TItle:Punch Bowl Fruit Salad\n'
             '\n'
             'Ingredients: ["2 large cans sliced peaches", "2 large cans fruit '
             'cocktail", "1 large can diced pineapple", "1 qt. fresh '
             'strawberries", "2 red apples", "2 yellow apples", "4 large ripe '
             'bananas", "1 lb. white grapes", "1 lb. purple grapes"]\n'
             '\n'
             'Generic ingredients:',
  'role': 'system'}]


In [ ]:
completion = client.chat.completions.create(
    model = 'ft:gpt-4o-mini-2024-07-18:personal::9yudTkS6',
    messages = test_messages,
    temperature=0,
    max_tokens=500

)

print(completion.choices[0].message.content)

["peaches", "fruit cocktail", "pineapple", "fresh strawberries", "red apples", "yellow apples", "bananas", "white grapes", "purple grapes"]


Task1_0822. 특정 과제에 대하여 추론하는 모델을 Chat completion api를 이용해서 아래와 같이 생성하여 비교하세요.
- zero-shot
- few-shot
- fine-tuning